In [1]:
import cdsapi
c = cdsapi.Client()

In [9]:
days = [str(i).zfill(2) for i in range(1, 32)]
months = [str(i).zfill(2) for i in range(1, 13)]
years = [str(i) for i in range(2000, 2023)]
time = [str(i).zfill(2) + ':00' for i in range(0, 24, 4)]
area =  [6, -176, -34, -35]  # North, East, South, West

In [11]:
(len(days) * len(months) * len(years) * len(time)) / 54 * 112 / 1024 / 5.75

18.083333333333332

In [12]:
c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'grib',
        'month': months,
        'year': years,
        'day': days,
        'time': time,
        'variable': '2m_temperature',
        'area': area
    },
    'download.grib'
)

2023-11-22 21:39:33,648 INFO Welcome to the CDS
2023-11-22 21:39:33,649 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2023-11-22 21:39:33,902 INFO Request is queued
2023-11-22 22:52:35,084 INFO Request is running
2023-11-22 23:28:56,050 INFO Request is completed
2023-11-22 23:28:56,051 INFO Downloading https://download-0020.copernicus-climate.eu/cache-compute-0020/cache/data3/adaptor.mars.internal-1700704005.9385033-2432-7-be72d764-e461-4be6-907f-6ce915f49de7.grib to download.grib (8.5G)
2023-11-23 00:04:12,021 INFO Download rate 4.1M/s      


Result(content_length=9175908240,content_type=application/x-grib,location=https://download-0020.copernicus-climate.eu/cache-compute-0020/cache/data3/adaptor.mars.internal-1700704005.9385033-2432-7-be72d764-e461-4be6-907f-6ce915f49de7.grib)

In [6]:
import xarray as xr
import cfgrib

# Load the GRIB file
ds = xr.open_dataset('download.grib', engine='cfgrib')

print(ds)

# Access a specific variable
temperature = ds['t2m']
print(temperature)

<xarray.Dataset>
Dimensions:     (time: 50406, latitude: 161, longitude: 565)
Coordinates:
    number      int32 ...
  * time        (time) datetime64[ns] 2000-01-01 ... 2022-12-31T20:00:00
    step        timedelta64[ns] ...
    surface     float64 ...
  * latitude    (latitude) float64 6.0 5.75 5.5 5.25 ... -33.5 -33.75 -34.0
  * longitude   (longitude) float64 -176.0 -175.8 -175.5 ... -35.5 -35.25 325.0
    valid_time  (time) datetime64[ns] ...
Data variables:
    t2m         (time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2023-11-23T19:13 GRIB to CDM+CF via cfgrib-0.9.1...
<xarray.DataArray 't2m' (time: 50406, latitude: 161, longitude: 565)>
[4585181790 values wi

In [13]:
sao_paulo = temperature.where(temperature.latitude - (-23.533773) + temperature.longitude - (-46.625290) < 0.01)

MemoryError: Unable to allocate 17.1 GiB for an array with shape (50406, 161, 565) and data type float32

## **Encontrar cidades grandes do Brasil**

### **Dados das cidades**
- Encontrar as latitudes e longitudes das principais cidades brasileiras: rio, salvador, brasilia, bh, são paulo, manaus
- Encontrar a temperatura da cidade com base na latitude e longitude
  - **Série temporal de 2000 - 2023, de 4 em 4 horas, para todos os dias, da temperatura da cidade**

### **Dados do El Niño**

- Encontrar a latitude e longitude do El Niño
- Calcular a média das temperaturas na região do El Niño
  - **Série temporal de 2000 - 2032, de 4 em 4 horas, para todos os dias, da temperatura na região do El niño**

### **Obtendo dados de São Paulo**

In [23]:
import cdsapi
c = cdsapi.Client()

In [24]:
days = [str(i).zfill(2) for i in range(1, 32)]
months = [str(i).zfill(2) for i in range(1, 13)]
years = [str(i) for i in range(2000, 2023)]
time = [str(i).zfill(2) + ':00' for i in range(0, 24, 4)]
area = [-23, -47, -24, -45]

In [25]:
c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'grib',
        'month': months,
        'year': years,
        'day': days,
        'time': time,
        'variable': '2m_temperature',
        'area': area
    },
    'download_sao_paulo.grib'
)

2023-11-23 20:35:45,303 INFO Welcome to the CDS
2023-11-23 20:35:45,304 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2023-11-23 20:35:45,593 INFO Request is completed
2023-11-23 20:35:45,594 INFO Downloading https://download-0011-clone.copernicus-climate.eu/cache-compute-0011/cache/data7/adaptor.mars.internal-1700778255.0761623-8851-1-522a20ad-b3dd-49aa-bda0-8d8719ffd8df.grib to download_sao_paulo.grib (11.5M)
2023-11-23 20:35:54,257 INFO Download rate 1.3M/s   


Result(content_length=12097440,content_type=application/x-grib,location=https://download-0011-clone.copernicus-climate.eu/cache-compute-0011/cache/data7/adaptor.mars.internal-1700778255.0761623-8851-1-522a20ad-b3dd-49aa-bda0-8d8719ffd8df.grib)

In [26]:
import xarray as xr
import cfgrib

# Load the GRIB file
ds = xr.open_dataset('download_sao_paulo.grib', engine='cfgrib')

print(ds)

# Access a specific variable
temperature = ds['t2m']
print(temperature)

<xarray.Dataset>
Dimensions:     (time: 50406, latitude: 5, longitude: 9)
Coordinates:
    number      int32 ...
  * time        (time) datetime64[ns] 2000-01-01 ... 2022-12-31T20:00:00
    step        timedelta64[ns] ...
    surface     float64 ...
  * latitude    (latitude) float64 -23.0 -23.25 -23.5 -23.75 -24.0
  * longitude   (longitude) float64 -47.0 -46.75 -46.5 ... -45.5 -45.25 315.0
    valid_time  (time) datetime64[ns] ...
Data variables:
    t2m         (time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2023-11-23T20:37 GRIB to CDM+CF via cfgrib-0.9.1...
<xarray.DataArray 't2m' (time: 50406, latitude: 5, longitude: 9)>
[2268270 values with dtype=float32]
Coor

In [47]:
import pandas 

In [28]:
sao_paulo = temperature.where(temperature.latitude - (-23.533773) + temperature.longitude - (-46.625290) < 0.01)

In [40]:
df = sao_paulo.to_dataframe().reset_index()

In [41]:
df_sao_paulo_temperatures = df[df['t2m'].notna()]

In [43]:
df_sao_paulo_temperatures = df_sao_paulo_temperatures.groupby('time').agg({'t2m': 'mean'}).reset_index()

In [44]:
df_sao_paulo_temperatures

,time,t2m
0,2000-01-01 00:00:00,295.405792
1,2000-01-01 04:00:00,294.166840
2,2000-01-01 08:00:00,294.208496
3,2000-01-01 12:00:00,295.512909
4,2000-01-01 16:00:00,296.298340
...,...,...
50401,2022-12-31 04:00:00,293.031555
50402,2022-12-31 08:00:00,292.620880
50403,2022-12-31 12:00:00,295.308746
50404,2022-12-31 16:00:00,298.525208


In [48]:
df_sao_paulo_temperatures.plot()

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

## **Análises de Séries Temporais**

- Verificar estacionariedade
- Verificar sazonalidade
- Computar o ACF (Autocorrelation Function) e PACF (Partial Autocorrelation Function)

## **ML**

### **Primeiro passo**

Utilizar modelos de Série Temporal para prever a temperatura de uma cidade com base exclusivamente na temperatura histórica de cada cidade.

### **Unindo os dados**

Utilizar modelos de Série Temporal multi-variado para agregar as informações da série temporal da cidade e a série temporal histórica do El Niño, para prever a temperatura de cada cidade.

## **Possíveis conclusões**

- Iremos observar se é possível utilizar os dados históricos do El Niño para agregar como feature para a predição da temperatura de cidades brasileiras
- Rankear as cidades em que a previsão da temperatura foi impactada positivamente pelo utilização dessa outra série temporal